In [1]:
import os 
from openai import Client
import json
import pandas as pd

In [2]:
with open("docs_with_q_4o-mini.json", 'rt') as f_in:
    list = json.load(f_in)

In [3]:
len(list)

661

In [4]:
client = Client(api_key=os.environ["IA"])

In [5]:
list[1]

{'source': 'https://www.reddit.com/r/germany/wiki/citizenship',
 'content': '*By u/staplehill*\nGerman citizenship is [not based on where you are born](https://en.wikipedia.org/wiki/Jus_soli). You get German citizenship either through naturalization after living in Germany for a number of years, or you inherit it from a parent. German citizenship *can* be passed down through multiple generations who live abroad and have no idea that they are actually German citizens. But one can also lose German citizenship just as unknowingly, for example by voluntarily getting the citizenship of another country (1914-today), or due to sex-discriminatory laws that in the past often allowed only the father to pass on German citizenship to a child. If German citizenship got lost due to sex discrimination then there may now be a pathway to restore it - but often not if it was lost for other reasons.\nDo you have a German ancestor and want to know if you can get a German passport? In order to find out, we

In [6]:
prompt_template = """
You're a RAG project assistant. Generate 5 question user might ask based on the FAQ record. The record should contain the answer to the questions, and the questions should be complete and not too short.
If possible, use as fewer words as possible from the record. 

The record:

Headline: {headline}
Question: {question}
Content: {content}


Provide the output in parsable JSON without using code blocks:

["question1", "question2", ..., "question5"]
""".strip()

In [7]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [8]:
test = generate_questions(list[1])

In [9]:
from tqdm.auto import tqdm

In [10]:
json.loads(test)

['What methods are available to obtain German citizenship if I have German ancestors?',
 "How can I determine if I am eligible for German citizenship based on my family's history?",
 'What factors can lead to losing German citizenship for individuals with German heritage?',
 'Are there specific gender-related laws that have historically affected the inheritance of German citizenship?',
 'What should I do if I want to explore my eligibility for a German passport based on my ancestry?']

In [11]:
results = {}

In [12]:
for doc in tqdm(list):
    doc_id = doc["id"]
    questions = generate_questions(doc)
    results[doc_id] = questions


  0%|          | 0/661 [00:00<?, ?it/s]

In [13]:
len(results)

661

In [14]:
index =  {d["id"]: d for d in list}

In [15]:
final_q = []

In [16]:
parsed_resulst = {}

for doc_id, json_questions in results.items():
    parsed_resulst[doc_id] = json.loads(json_questions)


In [17]:
for doc_id, questions in parsed_resulst.items():
    headline = index[doc_id]["headline"]
    for q in questions:
        final_q.append((q, headline, doc_id))

In [18]:
final_q[2]

('What were the safety statistics for US interstate highways in comparison to the Autobahn?',
 'How safe is the Autobahn?',
 '9d8370cf-a2c8-4c54-9f9c-476b9c09a933')

In [19]:
df = pd.DataFrame(final_q, columns=['question', 'headline', 'content'])
len(df)

3305

In [20]:
df.to_csv("./gp4o-mini-questions.csv", index=False)